# 쓰레기 분류 CNN 모델링

## 1. 라이브러리 및 모듈 임포트

In [ ]:
import os
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

## 2. 데이터 준비

In [ ]:
# 데이터셋 다운로드 및 압축 해제
dataset_url = "https://github.com/ardamavi/garbage-classification/raw/master/dataset.zip"
zip_path = tf.keras.utils.get_file(
    'garbage_classification.zip',
    origin=dataset_url,
    extract=True,
    archive_format='zip'
)
# 실제 데이터 디렉토리 경로 설정
base_dir = pathlib.Path(zip_path).parent
data_dir = base_dir / 'dataset'
print(f"데이터셋 경로: {data_dir}")

# 'trash' 폴더가 문제를 일으킬 수 있으므로 'misc'로 이름 변경
trash_dir = data_dir / 'trash'
misc_dir = data_dir / 'misc'
if trash_dir.exists() and not misc_dir.exists():
    os.rename(trash_dir, misc_dir)
    print(f"'{trash_dir.name}' 폴더를 '{misc_dir.name}'로 변경했습니다.")

# 클래스(폴더) 이름 확인
class_names = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])
print(f"클래스 종류: {class_names}")

# 데이터셋에 포함된 총 이미지 수 확인
image_count = len(list(data_dir.glob('*/*.jpg'))) + len(list(data_dir.glob('*/*.png')))
print(f"총 이미지 개수: {image_count}")

## 3. 데이터 로딩 및 전처리

In [ ]:
# 하이퍼파라미터 설정
batch_size = 32
img_height = 180
img_width = 180

# 훈련 데이터셋 로드 (80%)
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# 검증 데이터셋 로드 (20%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# 클래스 이름 확인
class_names = train_ds.class_names
print(f"데이터셋 클래스: {class_names}")

## 4. CNN 모델 정의 및 컴파일

In [ ]:
num_classes = len(class_names)

model = models.Sequential([
    # 입력 이미지의 픽셀 값을 [0, 1] 범위로 정규화
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    
    # Convolutional Block 1
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    # Convolutional Block 2
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    # Convolutional Block 3
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    # 과적합 방지를 위한 Dropout
    layers.Dropout(0.2),
    
    # Fully Connected Layer
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax') # 출력층
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# 모델 구조 출력
model.summary()

## 5. 모델 학습

In [ ]:
epochs = 15
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

## 6. 학습 결과 시각화

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 7. 모델 평가

In [ ]:
print("검증 데이터셋으로 모델 성능을 평가합니다...")
loss, acc = model.evaluate(val_ds, verbose=2)
print(f"\n최종 검증 정확도: {acc*100:.2f}%")